In [1]:
!pip install transformers torch pandas ipywidgets
print("Packages installed successfully!")


Packages installed successfully!


In [2]:
import torch
import pandas as pd
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    print("Nice! GPU is available 🔥")
    print("Using:", torch.cuda.get_device_name(0))
else:
    print("No GPU found 😓 Using CPU for now")

device = 0 if torch.cuda.is_available() else -1

print("🔄 Loading AI models...")
text_generator = pipeline("text-generation", model="gpt2-medium", device=device)
print("✅ GPT-2 model loaded!")

try:
    fake_news_detector = pipeline(
        "text-classification",
        model="hamzab/roberta-fake-news-classification",
        device=device
    )
    print("✅ Fake News Detector model loaded!")
except:
    print("⚠️ Could not load fake news detection model, using fallback.")


Nice! GPU is available 🔥
Using: Tesla T4
🔄 Loading AI models...


Device set to use cuda:0


✅ GPT-2 model loaded!


Device set to use cuda:0


✅ Fake News Detector model loaded!


In [3]:
from IPython.display import display, HTML
import uuid

unique_id = str(uuid.uuid4()).replace("-", "")

html_template = """
<style>
    .news-ui-container {{
        max-width: 700px;
        margin: 30px auto;
        padding: 20px;
        border-radius: 15px;
        background: linear-gradient(to right, #f8f9fa, #e9ecef);
        box-shadow: 0 4px 10px rgba(0,0,0,0.15);
        font-family: Arial, sans-serif;
    }}
    .news-ui-title {{
        text-align: center;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 15px;
    }}
    .news-ui-textarea {{
        width: 100%;
        height: 120px;
        padding: 10px;
        font-size: 16px;
        border-radius: 8px;
        border: 1px solid #ccc;
        resize: vertical;
    }}
    .news-ui-buttons {{
        display: flex;
        justify-content: space-between;
        margin-top: 15px;
    }}
    .news-ui-buttons button {{
        padding: 10px 18px;
        border: none;
        border-radius: 8px;
        background-color: #007bff;
        color: white;
        font-size: 15px;
        cursor: pointer;
        transition: background-color 0.3s ease;
    }}
    .news-ui-buttons button:hover {{
        background-color: #0056b3;
    }}
    .news-ui-output {{
        margin-top: 20px;
        padding: 15px;
        background-color: #fff3cd;
        border-left: 6px solid #ffeeba;
        border-radius: 10px;
        font-size: 16px;
    }}
</style>

<div class="news-ui-container">
    <div class="news-ui-title">📰 Fake News Detector & Generator</div>
    <textarea id="news_input_{0}" class="news-ui-textarea" placeholder="Enter your news text here..."></textarea>
    <div class="news-ui-buttons">
        <button onclick="detectFakeNews_{0}()">Detect Fake News</button>
        <button onclick="generateFakeNews_{0}()">Generate Fake News</button>
    </div>
    <div id="news_output_{0}" class="news-ui-output">Results will appear here.</div>
</div>

<script>
    function detectFakeNews_{0}() {{
        const text = document.getElementById('news_input_{0}').value;
        if (text.trim() === '') {{
            alert('Please enter some news text!');
            return;
        }}
        google.colab.kernel.invokeFunction(
            'notebook.detect_fake_news', [text], {{}}).then(output => {{
                document.getElementById('news_output_{0}').innerHTML = output.data['text/plain'];
            }});
    }}

    function generateFakeNews_{0}() {{
        const text = document.getElementById('news_input_{0}').value;
        if (text.trim() === '') {{
            alert('Please enter some prompt text!');
            return;
        }}
        google.colab.kernel.invokeFunction(
            'notebook.generate_fake_news', [text], {{}}).then(output => {{
                document.getElementById('news_output_{0}').innerHTML = output.data['text/plain'];
            }});
    }}
</script>
"""

display(HTML(html_template.format(unique_id)))


In [4]:
from google.colab import output

def detect_fake_news(text):
    try:
        result = fake_news_detector(text, truncation=True, max_length=512)
        label = result[0]['label']
        score = result[0]['score']
        percentage = round(score * 100, 2)
        response = f"<b>Prediction:</b> {label} <br><b>Confidence:</b> {percentage}%"
        return response
    except Exception as e:
        return f"Error during detection: {e}"

def generate_fake_news(text):
    try:
        result = text_generator(
            text, max_length=100, num_return_sequences=1, truncation=True
        )
        generated = result[0]['generated_text'].strip()
        # Clean up unwanted parts
        generated = generated.replace('\n', '<br>').replace('\xa0', ' ')
        if "ANI" in generated:
            generated = generated.split("ANI")[0]  # Remove ANI mention if needed
        response = f"<b>{generated}</b>"
        return response
    except Exception as e:
        return f"Error during generation: {e}"

output.register_callback('notebook.detect_fake_news', detect_fake_news)
output.register_callback('notebook.generate_fake_news', generate_fake_news)
